In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.express as px
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import requests
import pandas as pd

kubectl port-forward svc/scenario3 5000:5000  
kubectl port-forward svc/scenario4 5002:5000  
kubectl port-forward svc/scenario5 5001:5000  
kubectl port-forward service/router -n fission 9090:80  
kubectl port-forward service/elasticsearch-master -n elastic 9200:9200  

To run these visualizations successfully you need to run the commands above in separate terminals.

### TOPIC 1

In [11]:
platform_dropdown = widgets.Dropdown(
    options=[("All", "all"),
             ("Bluesky", "bluesky"),
             ("Mastodon", "mastodon"),
             ("Reddit", "reddit")],
    value="all",
    description="Platform:"
)

plot_type_dropdown = widgets.Dropdown(
    options=[("Histogram", "hist"),
             ("Boxplot", "box"),
             ("Both", "both")],
    value="both",
    description="Plot type:"
)

button = widgets.Button(description="Generate plot", button_style="info")
out = widgets.Output()

In [12]:
def plot_sentiment_diff(df, plot_type="both"):
    if df.empty:
        print("No data for current selection.")
        return
    if plot_type in ("hist", "both"):
        fig = px.histogram(df, x="sentiment_diff",
                           nbins=30,
                           title="Histogram – sentiment_diff")
        fig.show()
    if plot_type in ("box", "both"):
        fig = px.box(df, y="sentiment_diff",
                     points="all",
                     title="Boxplot – sentiment_diff")
        fig.show()

In [13]:
def on_click_generate_plot(b):
    print("✅ Button clicked!") 
    with out:
        clear_output(wait=True)
        platform = platform_dropdown.value
        ptype    = plot_type_dropdown.value

        url = f"http://localhost:9090/sentiment-diff/platform/{platform}/index/search_all"


        try:
            print(f"Fetching data from {url}")
            response = requests.get(url)
            response.raise_for_status()

            json_data = response.json()
            df = pd.DataFrame(json_data["data"])  
        except requests.RequestException as e:
            print(f"❌ API Error: {e}")
            return
        
        except ValueError as e:
            print(e)
            return
        except Exception as e:
            print(f"❌ Error fetching data: {e}")
            return
        
        if df.empty:
            print("No data returned from API.")
            return

        plot_sentiment_diff(df, plot_type=ptype)
try:
    button._click_handlers.callbacks.clear()
except AttributeError:
    pass

button.on_click(on_click_generate_plot)

In [14]:
display(widgets.HBox([platform_dropdown, plot_type_dropdown]), button, out)

Button(button_style='info', description='Generate plot', style=ButtonStyle())

Output()

### TOPIC 2

In [27]:
start_picker  = widgets.DatePicker(description="Start")
end_picker    = widgets.DatePicker(description="End")
k_dropdown    = widgets.Dropdown(options=[3,5,10], value=5, description="Top-K")
platform_dd   = widgets.Dropdown(
    options=[("All", "all"),
             ("Bluesky", "bluesky"),
             ("Mastodon", "mastodon"),
             ("Reddit", "reddit")],
    value="all",
    description="Platform"
)
run_btn = widgets.Button(description="Generate", button_style="info")
out     = widgets.Output()


In [28]:
def on_generate(b):
    with out:
        clear_output(wait=True)
        if not start_picker.value or not end_picker.value:
            print("Please select Start & End date"); 
            return

        start = start_picker.value.isoformat()
        end   = end_picker.value.isoformat()
        k     = k_dropdown.value
        plat  = platform_dd.value

        url = f"http://localhost:9090/sentiment-trends/start/{start}/end/{end}/k/{k}/platform/{plat}/index/search_all"

        try:
            print(f"Fetching data from {url}")
            response = requests.get(url)
            response.raise_for_status()
            json_data = response.json()
            df = pd.DataFrame(json_data["data"])
        except requests.RequestException as e:
            print(f"❌ API Error: {e}")
            return
        except ValueError as e:
            print(e); return
        if df.empty:
            print("No data returned from API.")
            return

        fig = px.line(df, x="timestamp", y="score",
                      color="post_id",
                      title=f"Top {k} sentiment trend – {plat}")
        fig.update_layout(legend=dict(orientation="h", yanchor="bottom", 
                                      y=1.1, xanchor="center", x=0.8),
                          legend_title_text="post_id")

        fig.show()

run_btn.on_click(on_generate)


In [29]:
display(widgets.HBox([start_picker, end_picker, k_dropdown, platform_dd, run_btn]), out)

Output()

### TOPIC 3

In [2]:
start_d   = widgets.DatePicker(description="Start")
end_d     = widgets.DatePicker(description="End")
platforms = ["all", "bluesky", "mastodon", "reddit"]
plat_dd   = widgets.Dropdown(options=platforms, value="all", description="Platform")
run_btn   = widgets.Button(description="Generate", button_style="info")
out       = widgets.Output()

In [4]:
def on_generate(b):
    with out:
        clear_output(wait=True)
        if not start_d.value or not end_d.value:
            print("Please select Start & End date")
            return

        start = start_d.value.isoformat()
        end   = end_d.value.isoformat()
        plat  = plat_dd.value

        payload = {
            "index_name": "search_all",
            "start": start,
            "end": end,
            "platform": plat
        }

        try:
            response = requests.post("http://localhost:5000/scenario3", json=payload)
            response.raise_for_status()
            result = response.json()

            if "error" in result:
                print(result["error"])
                return

            df = pd.DataFrame(result)
            if df.empty:
                print(" No data returned.")
                return

            # Plot
            fig = px.line(df, x="date", y="avg_score",
                          title=f"Trump daily sentiment – {plat}",
                          markers=True)
            
            fig.update_layout(
                yaxis_title="Average sentiment",
                              xaxis_title="Date"
            )
            fig.show()

        except Exception as e:
            print("Error:", e)

run_btn.on_click(on_generate)

In [5]:
display(widgets.HBox([start_d, end_d, plat_dd, run_btn]), out)

Output()

### TOPIC 4

In [ ]:
# Platform Summary (Average Sentiment / Std / Post Volume)
start_dp = widgets.DatePicker(description="Start")
end_dp   = widgets.DatePicker(description="End")
btn_sum  = widgets.Button(description="Generate Summary", button_style="info")
out_sum  = widgets.Output()

In [ ]:
def gen_summary(_):
    with out_sum:
        clear_output(wait=True)
        start = start_dp.value.isoformat() if start_dp.value else None
        end   = end_dp.value.isoformat()   if end_dp.value else None

        try:
            res = requests.post("http://localhost:5002/platform_summary", json={
                "start": start,
                "end": end
            })
            res.raise_for_status()
            data = res.json()

            if "error" in data or not data:
                print("No data or error returned:", data.get("error", "No records found."))
                return

            df = pd.DataFrame(data)

            # Plot 1: Average sentiment ± std
            fig1 = px.bar(df, x="platform", y="avg",
                          error_y="std",
                          title="Average sentiment ± std")

            # Plot 2: Post volume
            fig2 = px.bar(df, x="platform", y="count",
                          title="Post volume")

            fig1.show()
            fig2.show()

        except Exception as e:
            print(f"Error: {e}")

btn_sum.on_click(gen_summary)

In [ ]:
display(widgets.HBox([start_dp, end_dp, btn_sum]), out_sum)

In [ ]:
start_dp = widgets.DatePicker(description="Start")
end_dp   = widgets.DatePicker(description="End")
plat_dd  = widgets.Dropdown(options=["reddit", "mastodon", "bluesky"],
                      value="reddit", description="Platform")
win_sl   = widgets.IntSlider(value=7, min=3, max=30, step=1,
                       description="Window (days)")
btn_roll = widgets.Button(description="Rolling Plot", button_style="info")
out_roll = widgets.Output()

In [ ]:
def gen_roll(b):
    with out_roll:
        clear_output(wait=True)

        platform = plat_dd.value
        window = win_sl.value
        start = start_dp.value.isoformat() if start_dp.value else None
        end = end_dp.value.isoformat() if end_dp.value else None

        try:
            res = requests.post("http://localhost:5002/rolling_health", json={
                "platform": platform,
                "window_days": window,
                "start": start,
                "end": end
            })
            res.raise_for_status()
            data = res.json()

            if "error" in data or not data:
                print("No data or error returned:", data.get("error", "No records found."))
                return

            df = pd.DataFrame(data)

            df["date"] = pd.to_datetime(df["date"])  # ← ensure it's not a string

            fig = px.line(df, x="date", y="rolling_avg",
                        title=f"{platform}: {window}-day rolling avg sentiment",
                        markers=True)

            fig.add_scatter(x=df["date"], y=df["rolling_std"],
                            mode="lines", name="rolling_std",
                            line=dict(color="orange", dash="dash"))

            fig.update_layout(
                yaxis_title="Score / Std",
                xaxis_title="Date",
                xaxis_tickformat="%b %Y",  # <-- this fixes the clutter
                xaxis_tickangle=0
            )

            fig.show()
        except Exception as e:
            print(f"Error: {e}")
            

btn_roll.on_click(gen_roll)

In [ ]:
display(widgets.HBox([start_dp, end_dp,plat_dd, win_sl, btn_roll]), out_roll)

### TOPIC 5

In [ ]:
start_d  = widgets.DatePicker(description="Start")
end_d    = widgets.DatePicker(description="End")
plat_dd  = widgets.Dropdown(options=["all", "reddit", "mastodon", "bluesky"],
                      value="all", description="Platform")
ngram_dd = widgets.Dropdown(options=[("Unigram",1), ("Bigram",2), ("Trigram",3)],
                      value=1, description="N-gram")
method_dd = widgets.Dropdown(options=[("Frequency","freq"),
                                ("TF-IDF","tfidf")],
                       value="freq", description="Scoring")
slice_dd = widgets.Dropdown(options=[("All posts",None),
                               ("Positive 25 %","pos"),
                               ("Negative 25 %","neg")],
                      value=None, description="Sent slice")
top_n_s  = widgets.IntSlider(value=20, min=10, max=100, step=10,
                       description="Top-N")
btn      = widgets.Button(description="Generate Word-Cloud",
                    button_style="info")
out      = widgets.Output()

In [ ]:
def on_click(_):
    with out:
        clear_output(wait=True)
        if not start_d.value or not end_d.value:
            print("Please select the start and end date"); return

        # Prepare API payload
        payload = {
            "index_name": "search_all",
            "start": start_d.value.isoformat(),
            "end": end_d.value.isoformat(),
            "platform": plat_dd.value,
            "top_n": top_n_s.value,
            "ngram": ngram_dd.value,
            "method": method_dd.value,
            "sentiment_slice": slice_dd.value
        }

        try:
            response = requests.post("http://localhost:5001/scenario5", json=payload)
            print("Response text:", response.text)
            response.raise_for_status()
            data = response.json()
            
            if "error" in data:
                print(data["error"])
                return

            if not data:
                print("No results found.")
                return

            # Convert list of {"term": ..., "score": ...} to a dict
            freqs = {item["term"]: item["score"] for item in data}

            wc = WordCloud(width=900, height=450, background_color="white")
            wc.generate_from_frequencies(freqs)

            plt.figure(figsize=(11,5))
            plt.imshow(wc, interpolation="bilinear")
            plt.axis("off")
            title = (f"{plat_dd.value}  {ngram_dd.label}  {method_dd.label}  "
                     f"{'('+slice_dd.label+')' if slice_dd.value else ''}\n"
                     f"{start_d.value} → {end_d.value}")
            plt.title(title)
            plt.show()

        except Exception as e:
            print( str(e))
btn.on_click(on_click)

In [ ]:
display(
    widgets.VBox([
        widgets.HBox([start_d, end_d]),
        widgets.HBox([plat_dd, ngram_dd, method_dd, slice_dd, top_n_s]),
        btn, out
    ])
)